In [17]:
%load_ext autoreload
%autoreload 2

from datasets import load_dataset, load_from_disk, Dataset
from datasets.distributed import split_dataset_by_node
from src.maxdiffusion.input_pipeline.input_pipeline_interface import load_as_tf_dataset


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# def load_as_tf_dataset(dataset, batch_size, shuffle):
#   dataset = dataset.with_format("tensorflow")[:]
#   tf_dataset = tf.data.Dataset.from_tensor_slices(dataset)

#   if shuffle:
#     tf_dataset = tf_dataset.shuffle(len(tf_dataset))
#   tf_dataset = tf_dataset.batch(1, drop_remainder=True)
#   tf_dataset = tf_dataset.prefetch(tf.data.experimental.AUTOTUNE)
#   tf_dataset = tf_dataset.repeat(-1)

#   return tf_dataset

In [17]:
#dataset_save_location="gs://jfacevedo-maxdiffusion-v5p/pokemon-datasets/pokemon-gpt4-captions_xl"
dataset_save_location="/tmp/pokemon-gpt4-captions_xl/pokemon-datasets/pokemon-gpt4-captions_xl"
global_batch_size=4
train_ds = load_from_disk(dataset_save_location)
train_ds = load_as_tf_dataset(
    train_ds, global_batch_size, True
)

In [19]:
for x in train_ds:
    batch = x
    print(x)
    break

{'prompt_embeds': <tf.Tensor: shape=(4, 77, 2048), dtype=float32, numpy=
array([[[-3.8750000e+00, -2.5312500e+00,  4.7500000e+00, ...,
          1.8554688e-01,  4.2187500e-01, -2.9687500e-01],
        [ 8.6425781e-02, -6.1328125e-01, -4.8242188e-01, ...,
          5.0390625e-01, -3.7841797e-02, -1.6113281e-01],
        [-2.5512695e-02,  1.5312500e+00, -3.1054688e-01, ...,
          1.0078125e+00, -9.2968750e-01, -7.1093750e-01],
        ...,
        [ 2.2851562e-01,  1.6093750e+00, -6.0937500e-01, ...,
         -8.4765625e-01, -5.6640625e-01,  5.5468750e-01],
        [ 2.2070312e-01,  1.6250000e+00, -6.0546875e-01, ...,
         -7.8125000e-01, -5.5078125e-01,  6.0546875e-01],
        [ 1.8652344e-01,  1.5781250e+00, -5.7031250e-01, ...,
         -8.2812500e-01, -4.9804688e-01,  5.3125000e-01]],

       [[-3.8750000e+00, -2.5312500e+00,  4.7500000e+00, ...,
          1.8554688e-01,  4.2187500e-01, -2.9687500e-01],
        [ 8.6425781e-02, -6.1328125e-01, -4.8242188e-01, ...,
          

In [21]:
batch.keys()

dict_keys(['prompt_embeds', 'text_embeds', 'input_ids', 'pixel_values'])

In [24]:
batch['prompt_embeds'][0]

<tf.Tensor: shape=(77, 2048), dtype=float32, numpy=
array([[-3.875     , -2.53125   ,  4.75      , ...,  0.18554688,
         0.421875  , -0.296875  ],
       [ 0.08642578, -0.61328125, -0.48242188, ...,  0.50390625,
        -0.0378418 , -0.16113281],
       [-0.0255127 ,  1.53125   , -0.31054688, ...,  1.0078125 ,
        -0.9296875 , -0.7109375 ],
       ...,
       [ 0.22851562,  1.609375  , -0.609375  , ..., -0.84765625,
        -0.56640625,  0.5546875 ],
       [ 0.22070312,  1.625     , -0.60546875, ..., -0.78125   ,
        -0.55078125,  0.60546875],
       [ 0.18652344,  1.578125  , -0.5703125 , ..., -0.828125  ,
        -0.49804688,  0.53125   ]], dtype=float32)>

In [25]:
batch['text_embeds'][0]

<tf.Tensor: shape=(1280,), dtype=float32, numpy=
array([ 0.6171875 ,  1.0625    ,  0.21484375, ...,  0.56640625,
       -1.3984375 , -0.6484375 ], dtype=float32)>

In [2]:
dataset_name="diffusers/pokemon-gpt4-captions"
train_ds2 = load_dataset(dataset_name,split="train",streaming=True)

In [18]:
train_ds2 = train_ds2.batch(batch_size=2, drop_last_batch=True)
train_ds2 = split_dataset_by_node(train_ds2, world_size=4, rank=0)
train_ds2 = train_ds2.with_format("np")

In [19]:
for x in train_ds2:
    batch2 = x
    print(x)
    break

{'image': array([array([[[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
               [255, 255, 255],
               [255, 255, 255],
               [255, 255, 255]],

              [[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
               [255, 255, 255],
               [255, 255, 255],
               [255, 255, 255]],

              [[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
               [255, 255, 255],
               [255, 255, 255],
               [255, 255, 255]],

              ...,

              [[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
               [255, 255, 255],
               [255, 255, 255],
               [255, 255, 255]],

              [[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...

In [20]:
train_iter = iter(train_ds2)
next(train_iter)

{'image': array([array([[[255, 255, 255],
                [255, 255, 255],
                [255, 255, 255],
                ...,
                [255, 255, 255],
                [255, 255, 255],
                [255, 255, 255]],
 
               [[255, 255, 255],
                [255, 255, 255],
                [255, 255, 255],
                ...,
                [255, 255, 255],
                [255, 255, 255],
                [255, 255, 255]],
 
               [[255, 255, 255],
                [255, 255, 255],
                [255, 255, 255],
                ...,
                [255, 255, 255],
                [255, 255, 255],
                [255, 255, 255]],
 
               ...,
 
               [[255, 255, 255],
                [255, 255, 255],
                [255, 255, 255],
                ...,
                [255, 255, 255],
                [255, 255, 255],
                [255, 255, 255]],
 
               [[255, 255, 255],
                [255, 255, 255],
               

In [7]:
import tensorflow as tf
tf_dataset = train_ds2.with_format("tensorflow")
#tf_dataset = tf.data.Dataset.from_tensor_slices(dataset)
tf_dataset = tf_dataset.shuffle(len(tf_dataset))
tf_dataset = tf_dataset.batch(batch_size // jax.process_count(), drop_remainder=True)
tf_dataset = tf_dataset.prefetch(tf.data.experimental.AUTOTUNE)
tf_dataset = tf_dataset.repeat(-1)

In [8]:
for x in tf_dataset:
    print(x)
    break

{'image': <tf.Tensor: shape=(1280, 1280, 3), dtype=uint8, numpy=
array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [

In [10]:
from src.maxdiffusion.transformers import CLIPTokenizer
image_column='image'
caption_column='text'
tokenizer_config = {"path" : "/tmp/checkpoints/models--stabilityai--stable-diffusion-xl-base-1.0"}
tokenizer_path="/home/aireenmei/checkpoints/models--stabilityai--stable-diffusion-xl-base-1.0/tokenizer"
tokenizer_path_2="/home/aireenmei/checkpoints/models--stabilityai--stable-diffusion-xl-base-1.0/tokenizer_2"
activations_dtype='bfloat16'
weights_dtype='float32'

tokenizer = CLIPTokenizer.from_pretrained(
                tokenizer_path,
                #subfolder="tokenizer",
                dtype=activations_dtype,
                weights_dtype=weights_dtype
            )

tokenizer_2 = CLIPTokenizer.from_pretrained(
                tokenizer_path_2,
                #subfolder="tokenizer_2",
                dtype=activations_dtype,
                weights_dtype=weights_dtype
            )


In [12]:
from functools import partial
# sdxl_trainer
def tokenize_captions_xl(examples, caption_column, tokenizers, p_encode=None):
    inputs = []
    captions = list(examples[caption_column])
    for _tokenizer in tokenizers:
        text_inputs = _tokenizer(
            captions,
            padding="max_length",
            max_length=_tokenizer.model_max_length,
            truncation=True,
            return_tensors="np"
        )
        inputs.append(text_inputs.input_ids)
    inputs = np.stack(inputs,axis=1)

    if p_encode:
        prompt_embeds, text_embeds = p_encode(inputs)
        # pyarrow dataset doesn't support bf16, so cast to float32.
        examples["prompt_embeds"] = np.float32(prompt_embeds)
        examples["text_embeds"] = np.float32(text_embeds)
    examples["input_ids"] = inputs
    return examples

# sdxl_trainer
tokenize_fn = partial(tokenize_captions_xl,
                caption_column=caption_column,
                tokenizers=[tokenizer, tokenizer_2],
                p_encode=p_encode)

def encode_xl(input_ids, text_encoders, text_encoder_params):
    te_1_inputs = input_ids[:, 0, :]
    te_2_inputs = input_ids[:, 1, :]

    prompt_embeds = text_encoders[0](
        te_1_inputs, params=text_encoder_params[0], output_hidden_states=True
    )
    prompt_embeds = prompt_embeds["hidden_states"][-2]

    prompt_embeds_2_out = text_encoders[1](
        te_2_inputs, params=text_encoder_params[1], output_hidden_states=True
    )
    prompt_embeds_2 = prompt_embeds_2_out["hidden_states"][-2]
    text_embeds = prompt_embeds_2_out["text_embeds"]
    prompt_embeds = jnp.concatenate([prompt_embeds, prompt_embeds_2], axis=-1)

    return prompt_embeds, text_embeds

p_encode = partial(encode_xl,
            text_encoders=[pipeline.text_encoder, pipeline.text_encoder_2],
            text_encoder_params=[train_states["text_encoder_state"].params, train_states["text_encoder_2_state"].params])

train_ds = train_ds.map(
  function=tokenize_fn,
  batched=True,
  remove_columns=[captions_column],
  num_proc=1,
  desc="Running tokenizer on train dataset",
)

NameError: name 'p_encode' is not defined